In [ ]:
GAN 的简要实现流程如下所示。
(1) generator网络将形状为(latent_dim,)的向量映射到形状为(32, 32, 3)的图像。
(2) discriminator 网络将形状为 (32, 32, 3) 的图像映射到一个二进制分数，用于评
估图像为真的概率。
(3) gan 网络将 generator 网络和 discriminator 网络连接在一起： gan(x) = discriminator
(generator(x))。生成器将潜在空间向量解码为图像，判别器对这些图像的真实性进
行评估，因此这个 gan 网络是将这些潜在向量映射到判别器的评估结果。
(4) 我们使用带有“真” /“假”标签的真假图像样本来训练判别器，就和训练普通的图像
分类模型一样。
(5) 为了训练生成器，我们要使用 gan 模型的损失相对于生成器权重的梯度。这意味着，
在每一步都要移动生成器的权重，其移动方向是让判别器更有可能将生成器解码的图像
划分为“真”。换句话说，我们训练生成器来欺骗判别器。

下面是本节实现 GAN 生成器和判别器时用到的一些技巧。这里并没有列出与 GAN 相关的
全部技巧，更多技巧可查阅关于 GAN 的文献。
 我们使用 tanh 作为生成器最后一层的激活，而不用 sigmoid，后者在其他类型的模型中
更加常见。
 我们使用正态分布（高斯分布）对潜在空间中的点进行采样，而不用均匀分布。
 随机性能够提高稳健性。训练GAN得到的是一个动态平衡，所以GAN可能以各种方式“卡
住”。在训练过程中引入随机性有助于防止出现这种情况。我们通过两种方式引入随机性：
一种是在判别器中使用 dropout，另一种是向判别器的标签添加随机噪声。
 稀疏的梯度会妨碍 GAN 的训练。在深度学习中，稀疏性通常是我们需要的属性，但在
GAN 中并非如此。有两件事情可能导致梯度稀疏：最大池化运算和 ReLU 激活。我们推
荐使用步进卷积代替最大池化来进行下采样，还推荐使用 LeakyReLU 层来代替 ReLU 激
活。 LeakyReLU 和 ReLU 类似，但它允许较小的负数激活值，从而放宽了稀疏性限制。
 在生成的图像中，经常会见到棋盘状伪影，这是由生成器中像素空间的不均匀覆盖导致的
（见图 8-17）。为了解决这个问题，每当在生成器和判别器中都使用步进的 Conv2DTranpose
或 Conv2D 时，使用的内核大小要能够被步幅大小整除。

# GAN生成器网络

In [1]:
import keras
from keras import layers
import numpy as np

latent_dim = 32
height = 32
width = 32
channels = 3

generator_input = keras.Input(shape=(latent_dim,))
x = layers.Dense(128 * 16 * 16)(generator_input)
x = layers.LeakyReLU()(x)
x = layers.Reshape((16, 16, 128))(x)
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2DTranspose(256, 4, strides=2, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(channels, 7, activation='tanh', padding='same')(x)
generator = keras.models.Model(generator_input, x)
generator.summary()

Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 32768)             1081344   
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 32768)             0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 256)       819456    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 16, 16, 256)       0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 32, 32, 256)       1048832   
__________

# GAN判别器网络

In [2]:
discriminator_input = layers.Input(shape=(height, width, channels))
x = layers.Conv2D(128, 3)(discriminator_input)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Flatten()(x)
x = layers.Dropout(0.4)(x)
x = layers.Dense(1, activation='sigmoid')(x)
discriminator = keras.models.Model(discriminator_input, x)
discriminator.summary()
discriminator_optimizer = keras.optimizers.RMSprop(
                            lr=0.0008,
                            clipvalue=1.0,
                            decay=1e-8)
discriminator.compile(optimizer=discriminator_optimizer,
        loss='binary_crossentropy')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 30, 30, 128)       3584      
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 30, 30, 128)       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 14, 14, 128)       262272    
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 6, 6, 128)         262272    
_________________________________________________________________
leaky_re_lu_8 (LeakyReLU)    (None, 6, 6, 128)         0         
__________

# 对抗网络

In [3]:
discriminator.trainable = False
gan_input = keras.Input(shape=(latent_dim,))
gan_output = discriminator(generator(gan_input))
gan = keras.models.Model(gan_input, gan_output)
gan_optimizer = keras.optimizers.RMSprop(lr=0.0004, clipvalue=1.0, decay=1e-8)
gan.compile(optimizer=gan_optimizer, loss='binary_crossentropy')

# 实现GAN的训练

In [ ]:
现在开始训练。再次强调一下，训练循环的大致流程如下所示。每轮都进行以下操作。
(1) 从潜在空间中抽取随机的点（随机噪声）。
(2) 利用这个随机噪声用 generator 生成图像。
(3) 将生成图像与真实图像混合。
(4) 使用这些混合后的图像以及相应的标签（真实图像为“真”，生成图像为“假”）来训练
discriminator，如图 8-18 所示。
(5) 在潜在空间中随机抽取新的点。
(6) 使用这些随机向量以及全部是“真实图像”的标签来训练 gan。这会更新生成器的权重
（只更新生成器的权重，因为判别器在 gan 中被冻结），其更新方向是使得判别器能够
将生成图像预测为“真实图像”。这个过程是训练生成器去欺骗判别器。

In [4]:
import os
from keras.preprocessing import image
(x_train, y_train), (_, _) = keras.datasets.cifar10.load_data()
x_train = x_train[y_train.flatten() == 6]
x_train = x_train.reshape((x_train.shape[0],) +
(height, width, channels)).astype('float32') / 255.

iterations = 10000
batch_size = 20
save_dir = './gan/'

start = 0
for step in range(iterations):
    random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))
    generated_images = generator.predict(random_latent_vectors)
    stop = start + batch_size
    real_images = x_train[start: stop]
    combined_images = np.concatenate([generated_images, real_images])
    labels = np.concatenate([np.ones((batch_size, 1)), np.zeros((batch_size, 1))])
    labels += 0.05 * np.random.random(labels.shape)
    d_loss = discriminator.train_on_batch(combined_images, labels)
    random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))
    misleading_targets = np.zeros((batch_size, 1))
    a_loss = gan.train_on_batch(random_latent_vectors, misleading_targets)
    
    start += batch_size
    if start > len(x_train) - batch_size:
        start = 0
    if step % 100 == 0:
        gan.save_weights('gan.h5')
        print('discriminator loss:', d_loss)
        print('adversarial loss:', a_loss)
        img = image.array_to_img(generated_images[0] * 255., scale=False)
        img.save(os.path.join(save_dir, 'generated_frog' + str(step) + '.png'))
        img = image.array_to_img(real_images[0] * 255., scale=False)
        img.save(os.path.join(save_dir, 'real_frog' + str(step) + '.png'))

/home/gengfeng/anaconda3/lib/python3.6/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


discriminator loss: 0.700756
adversarial loss: 0.6949438
discriminator loss: 0.74460995
adversarial loss: 0.8886694
discriminator loss: 0.69181347
adversarial loss: 0.80976474
discriminator loss: 0.9451052
adversarial loss: 0.72260076
discriminator loss: 0.69935393
adversarial loss: 0.7079772
discriminator loss: 0.74927366
adversarial loss: 0.91789377
discriminator loss: 0.64778435
adversarial loss: 0.78957736
discriminator loss: 0.67937165
adversarial loss: 0.7707585
discriminator loss: 0.69006574
adversarial loss: 0.7503058
discriminator loss: 0.68111503
adversarial loss: 0.7335137
discriminator loss: 0.69570524
adversarial loss: 0.7673322
discriminator loss: 0.6885201
adversarial loss: 0.7679281
discriminator loss: 0.689068
adversarial loss: 0.7276888
discriminator loss: 0.6881999
adversarial loss: 0.75281554
discriminator loss: 0.6939681
adversarial loss: 0.76283354
discriminator loss: 0.7045405
adversarial loss: 0.8039566
discriminator loss: 0.6990553
adversarial loss: 0.7470444
d